In [1]:
import json

import scipy as sp
import pandas as pd
import numpy as np

from functools import partial
from math import sqrt

from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import confusion_matrix as sk_cmatrix
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from collections import Counter

import lightgbm as lgb
#A fast, distributed, high performance gradient boosting (GBDT, GBRT, GBM or MART) framework 
#based on decision tree algorithms, used for ranking, classification and many other machine learning tasks.
np.random.seed(369)

In [2]:
# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [3]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = quadratic_weighted_kappa(y, X_p)
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [4]:
def rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

In [5]:
#Load all CSV data, printing the rows and columns.
print('Train')
train = pd.read_csv("input/train/train.csv")
print(train.shape)

print('Test')
test = pd.read_csv("input/test/test.csv")
print(test.shape)

print('Breeds')
breeds = pd.read_csv("input/breed_labels.csv")
print(breeds.shape)

print('Colors')
colors = pd.read_csv("input/color_labels.csv")
print(colors.shape)

print('States')
states = pd.read_csv("input/state_labels.csv")
print(states.shape)

Train
(14720, 24)
Test
(3948, 23)
Breeds
(307, 3)
Colors
(7, 2)
States
(15, 2)


In [6]:
#set up arrays for the CSV columns that we want
target = train['AdoptionSpeed']
train_id = train['PetID']
test_id = test['PetID']
train.drop(['AdoptionSpeed', 'PetID'], axis=1, inplace=True)
test.drop(['PetID'], axis=1, inplace=True)

In [7]:
#Setup sentiment magnitudes and scores arrays
doc_sent_mag = []
doc_sent_score = []
nf_count = 0
for pet in train_id:
    try:
        with open('input/train/train_sentiment/' + '.json', 'r') as f:
            sentiment = json.load(f)
        doc_sent_mag.append(sentiment['documentSentiment']['magnitude'])
        doc_sent_score.append(sentiment['documentSentiment']['score'])
    except FileNotFoundError:
        nf_count += 1
        doc_sent_mag.append(-1)
        doc_sent_score.append(-1)

train.loc[:, 'doc_sent_mag'] = doc_sent_mag
train.loc[:, 'doc_sent_score'] = doc_sent_score

doc_sent_mag = []
doc_sent_score = []
nf_count = 0
for pet in test_id:
    try:
        with open('input/test/test_sentiment/' + '.json', 'r') as f:
            sentiment = json.load(f)
        doc_sent_mag.append(sentiment['documentSentiment']['magnitude'])
        doc_sent_score.append(sentiment['documentSentiment']['score'])
    except FileNotFoundError:
        nf_count += 1
        doc_sent_mag.append(-1)
        doc_sent_score.append(-1)

test.loc[:, 'doc_sent_mag'] = doc_sent_mag
test.loc[:, 'doc_sent_score'] = doc_sent_score

In [8]:
# sci-kit learn TFIDF Vectorizer is used for text feature extraction
# https://stackoverflow.com/questions/36800654/how-is-the-tfidfvectorizer-in-scikit-learn-supposed-to-work
# Truncated SVD is used to reduce dimensionality
train_desc = train.Description.fillna("none").values
test_desc = test.Description.fillna("none").values

tfv = TfidfVectorizer(min_df=2,  max_features=None,
        strip_accents='unicode', analyzer='word', token_pattern=r'(?u)\b\w+\b',
        ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1,
        )
    
# Fit TFIDF
tfv.fit(list(train_desc))
X =  tfv.transform(train_desc)
X_test = tfv.transform(test_desc)
print("X (tfidf):", X.shape)

svd = TruncatedSVD(n_components=200)
svd.fit(X)
# print(svd.explained_variance_ratio_.sum())
# print(svd.explained_variance_ratio_)
X = svd.transform(X)
print("X (svd):", X.shape)

# X = pd.DataFrame(X, columns=['svd_{}'.format(i) for i in range(120)])
# train = pd.concat((train, X), axis=1)
# X_test = svd.transform(X_test)
# X_test = pd.DataFrame(X_test, columns=['svd_{}'.format(i) for i in range(120)])
# test = pd.concat((test, X_test), axis=1)

print("train:", train.shape)

X (tfidf): (14720, 178520)
X (svd): (14720, 200)
train: (14720, 24)


In [9]:
## WITHOUT ERROR FIXED
train_desc = train.Description.fillna("none").values
test_desc = test.Description.fillna("none").values

tfv = TfidfVectorizer(min_df=3,  max_features=10000,
        strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
        ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1,
        stop_words = 'english')
    
# Fit TFIDF
tfv.fit(list(train_desc))
X =  tfv.transform(train_desc)
X_test = tfv.transform(test_desc)
print("X (tfidf):", X.shape)

svd = TruncatedSVD(n_components=120)
svd.fit(X)
# print(svd.explained_variance_ratio_.sum())
# print(svd.explained_variance_ratio_)
X = svd.transform(X)
print("X (svd):", X.shape)

X = pd.DataFrame(X, columns=['svd_{}'.format(i) for i in range(120)])
train = pd.concat((train, X), axis=1)
X_test = svd.transform(X_test)
X_test = pd.DataFrame(X_test, columns=['svd_{}'.format(i) for i in range(120)])
test = pd.concat((test, X_test), axis=1)

print("train:", train.shape)

X (tfidf): (14720, 10000)
X (svd): (14720, 120)
train: (14720, 144)


In [10]:
#load metadata
vertex_xs = []
vertex_ys = []
bounding_confidences = []
bounding_importance_fracs = []
dominant_blues = []
dominant_greens = []
dominant_reds = []
dominant_pixel_fracs = []
dominant_scores = []
label_descriptions = []
label_scores = []
nf_count = 0 #not found
nl_count = 0 #not labelled
for pet in train_id:
    try:
        with open('../input/train/train_metadata/' + '-1.json', 'r') as f:
            data = json.load(f)
        vertex_x = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['x']
        vertex_xs.append(vertex_x)
        vertex_y = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['y']
        vertex_ys.append(vertex_y)
        bounding_confidence = data['cropHintsAnnotation']['cropHints'][0]['confidence']
        bounding_confidences.append(bounding_confidence)
        bounding_importance_frac = data['cropHintsAnnotation']['cropHints'][0].get('importanceFraction', -1)
        bounding_importance_fracs.append(bounding_importance_frac)
        dominant_blue = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['blue']
        dominant_blues.append(dominant_blue)
        dominant_green = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['green']
        dominant_greens.append(dominant_green)
        dominant_red = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['red']
        dominant_reds.append(dominant_red)
        dominant_pixel_frac = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['pixelFraction']
        dominant_pixel_fracs.append(dominant_pixel_frac)
        dominant_score = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['score']
        dominant_scores.append(dominant_score)
        if data.get('labelAnnotations'):
            label_description = data['labelAnnotations'][0]['description']
            label_descriptions.append(label_description)
            label_score = data['labelAnnotations'][0]['score']
            label_scores.append(label_score)
        else:
            nl_count += 1
            label_descriptions.append('nothing')
            label_scores.append(-1)
    except FileNotFoundError:
        nf_count += 1
        vertex_xs.append(-1)
        vertex_ys.append(-1)
        bounding_confidences.append(-1)
        bounding_importance_fracs.append(-1)
        dominant_blues.append(-1)
        dominant_greens.append(-1)
        dominant_reds.append(-1)
        dominant_pixel_fracs.append(-1)
        dominant_scores.append(-1)
        label_descriptions.append('nothing')
        label_scores.append(-1)

print('train - not found: ', nf_count)
print('train - not labelled: ', nl_count)
train.loc[:, 'vertex_x'] = vertex_xs
train.loc[:, 'vertex_y'] = vertex_ys
train.loc[:, 'bounding_confidence'] = bounding_confidences
train.loc[:, 'bounding_importance'] = bounding_importance_fracs
train.loc[:, 'dominant_blue'] = dominant_blues
train.loc[:, 'dominant_green'] = dominant_greens
train.loc[:, 'dominant_red'] = dominant_reds
train.loc[:, 'dominant_pixel_frac'] = dominant_pixel_fracs
train.loc[:, 'dominant_score'] = dominant_scores
train.loc[:, 'label_description'] = label_descriptions
train.loc[:, 'label_score'] = label_scores


vertex_xs = []
vertex_ys = []
bounding_confidences = []
bounding_importance_fracs = []
dominant_blues = []
dominant_greens = []
dominant_reds = []
dominant_pixel_fracs = []
dominant_scores = []
label_descriptions = []
label_scores = []
nf_count = 0
nl_count = 0
for pet in test_id:
    try:
        with open('../input/test_metadata/' + '-1.json', 'r') as f:
            data = json.load(f)
        vertex_x = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['x']
        vertex_xs.append(vertex_x)
        vertex_y = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['y']
        vertex_ys.append(vertex_y)
        bounding_confidence = data['cropHintsAnnotation']['cropHints'][0]['confidence']
        bounding_confidences.append(bounding_confidence)
        bounding_importance_frac = data['cropHintsAnnotation']['cropHints'][0].get('importanceFraction', -1)
        bounding_importance_fracs.append(bounding_importance_frac)
        dominant_blue = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['blue']
        dominant_blues.append(dominant_blue)
        dominant_green = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['green']
        dominant_greens.append(dominant_green)
        dominant_red = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['red']
        dominant_reds.append(dominant_red)
        dominant_pixel_frac = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['pixelFraction']
        dominant_pixel_fracs.append(dominant_pixel_frac)
        dominant_score = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['score']
        dominant_scores.append(dominant_score)
        if data.get('labelAnnotations'):
            label_description = data['labelAnnotations'][0]['description']
            label_descriptions.append(label_description)
            label_score = data['labelAnnotations'][0]['score']
            label_scores.append(label_score)
        else:
            nl_count += 1
            label_descriptions.append('nothing')
            label_scores.append(-1)
    except FileNotFoundError:
        nf_count += 1
        vertex_xs.append(-1)
        vertex_ys.append(-1)
        bounding_confidences.append(-1)
        bounding_importance_fracs.append(-1)
        dominant_blues.append(-1)
        dominant_greens.append(-1)
        dominant_reds.append(-1)
        dominant_pixel_fracs.append(-1)
        dominant_scores.append(-1)
        label_descriptions.append('nothing')
        label_scores.append(-1)

print('test - not found: ', nf_count)
test.loc[:, 'vertex_x'] = vertex_xs
test.loc[:, 'vertex_y'] = vertex_ys
test.loc[:, 'bounding_confidence'] = bounding_confidences
test.loc[:, 'bounding_importance'] = bounding_importance_fracs
test.loc[:, 'dominant_blue'] = dominant_blues
test.loc[:, 'dominant_green'] = dominant_greens
test.loc[:, 'dominant_red'] = dominant_reds
test.loc[:, 'dominant_pixel_frac'] = dominant_pixel_fracs
test.loc[:, 'dominant_score'] = dominant_scores
test.loc[:, 'label_description'] = label_descriptions
test.loc[:, 'label_score'] = label_scores

train - not found:  14720
train - not labelled:  0
test - not found:  3948


In [11]:
train.drop(['Name', 'RescuerID', 'Description'], axis=1, inplace=True)
test.drop(['Name', 'RescuerID', 'Description'], axis=1, inplace=True)

In [12]:
numeric_cols = ['Age', 'Quantity', 'Fee', 'VideoAmt', 
                'PhotoAmt', 'AdoptionSpeed', 'doc_sent_mag', 
                'doc_sent_score', 'dominant_score', 'dominant_pixel_frac', 
                'dominant_red', 'dominant_green', 'dominant_blue', 
                'bounding_importance', 'bounding_confidence', 
                'vertex_x', 'vertex_y', 'label_score'] + ['svd_{}'.format(i) for i in range(120)]
cat_cols = list(set(train.columns) - set(numeric_cols))
train.loc[:, cat_cols] = train[cat_cols].astype('category')
test.loc[:, cat_cols] = test[cat_cols].astype('category')
print(train.shape)
print(test.shape)
train.head()

(14720, 152)
(3948, 152)


,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,...,vertex_y,bounding_confidence,bounding_importance,dominant_blue,dominant_green,dominant_red,dominant_pixel_frac,dominant_score,label_description,label_score
0,2,6,265,0,1,4,7,0,2,2,...,-1,-1,-1,-1,-1,-1,-1,-1,nothing,-1
1,1,2,307,0,2,1,2,7,2,2,...,-1,-1,-1,-1,-1,-1,-1,-1,nothing,-1
2,1,2,307,0,2,1,0,0,2,1,...,-1,-1,-1,-1,-1,-1,-1,-1,nothing,-1
3,2,2,241,241,1,1,4,6,2,1,...,-1,-1,-1,-1,-1,-1,-1,-1,nothing,-1
4,1,1,307,0,1,2,5,0,2,1,...,-1,-1,-1,-1,-1,-1,-1,-1,nothing,-1


In [13]:
# get the categorical features
foo = train.dtypes
cat_feature_names = foo[foo == "category"]
cat_features = [train.columns.get_loc(c) for c in train.columns if c in cat_feature_names]

In [15]:
def run_cv_model(train, test, target, model_fn, params={}, eval_fn=None, label='model'):
    kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    fold_splits = kf.split(train, target)
    cv_scores = []
    qwk_scores = []
    pred_full_test = 0
    pred_train = np.zeros((train.shape[0], 5))
    all_coefficients = np.zeros((5, 4))
    feature_importance_df = pd.DataFrame()
    i = 1
    for dev_index, val_index in fold_splits:
        print('Started ' + label + ' fold ' + str(i) + '/5')
        if isinstance(train, pd.DataFrame):
            dev_X, val_X = train.iloc[dev_index], train.iloc[val_index]
            dev_y, val_y = target[dev_index], target[val_index]
        else:
            dev_X, val_X = train[dev_index], train[val_index]
            dev_y, val_y = target[dev_index], target[val_index]
        params2 = params.copy()
        pred_val_y, pred_test_y, importances, coefficients, qwk = model_fn(dev_X, dev_y, val_X, val_y, test, params2)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index] = pred_val_y
        all_coefficients[i-1, :] = coefficients
        if eval_fn is not None:
            cv_score = eval_fn(val_y, pred_val_y)
            cv_scores.append(cv_score)
            qwk_scores.append(qwk)
            print(label + ' cv score {}: RMSE {} QWK {}'.format(i, cv_score, qwk))
        fold_importance_df = pd.DataFrame()
        fold_importance_df['feature'] = train.columns.values
        fold_importance_df['importance'] = importances
        fold_importance_df['fold'] = i
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)        
        i += 1
    print('{} cv RMSE scores : {}'.format(label, cv_scores))
    print('{} cv mean RMSE score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv std RMSE score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv QWK scores : {}'.format(label, qwk_scores))
    print('{} cv mean QWK score : {}'.format(label, np.mean(qwk_scores)))
    print('{} cv std QWK score : {}'.format(label, np.std(qwk_scores)))
    pred_full_test = pred_full_test / 5.0
    results = {'label': label,
               'train': pred_train, 'test': pred_full_test,
                'cv': cv_scores, 'qwk': qwk_scores,
               'importance': feature_importance_df,
               'coefficients': all_coefficients}
    return results

params = {'application': 'regression',
          'boosting': 'gbdt',
          'metric': 'rmse',
          'num_leaves': 70,
          'max_depth': 8,
          'learning_rate': 0.01,
          'bagging_fraction': 0.85,
          'feature_fraction': 0.8,
          'min_split_gain': 0.02,
          'min_child_samples': 150,
          'min_child_weight': 0.02,
          'lambda_l2': 0.0475,
          'verbosity': -1,
          'data_random_seed': 17,
          'early_stop': 100,
          'verbose_eval': 100,
          'num_rounds': 10000}

def runLGB(train_X, train_y, test_X, test_y, test_X2, params):
    print('Prep LGB')
    d_train = lgb.Dataset(train_X, label=train_y)
    d_valid = lgb.Dataset(test_X, label=test_y)
    watchlist = [d_train, d_valid]
    print('Train LGB')
    num_rounds = params.pop('num_rounds')
    verbose_eval = params.pop('verbose_eval')
    early_stop = None
    if params.get('early_stop'):
        early_stop = params.pop('early_stop')
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=num_rounds,
                      valid_sets=watchlist,
                      verbose_eval=verbose_eval,
                      categorical_feature=list(cat_features),
                      early_stopping_rounds=early_stop)
    
    print('Predict 1/2')
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    optR = OptimizedRounder()
    optR.fit(pred_test_y, test_y)
    coefficients = optR.coefficients()
    pred_test_y_k = optR.predict(pred_test_y, coefficients)
    print("Valid Counts = ", Counter(test_y))
    print("Predicted Counts = ", Counter(pred_test_y_k))
    print("Coefficients = ", coefficients)
    qwk = quadratic_weighted_kappa(test_y, pred_test_y_k)
    print("QWK = ", qwk)
    print('Predict 2/2')
    pred_test_y2 = model.predict(test_X2, num_iteration=model.best_iteration)
    return pred_test_y.reshape(-1, 1), pred_test_y2.reshape(-1, 1), model.feature_importance(), coefficients, qwk

results = run_cv_model(train, test, target, runLGB, params, rmse, 'lgb')

Started lgb fold 1/5
Prep LGB
Train LGB


D:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 150]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
D:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.05922	valid_1's rmse: 1.09822
[200]	training's rmse: 1.00397	valid_1's rmse: 1.07404
[300]	training's rmse: 0.969777	valid_1's rmse: 1.06407
[400]	training's rmse: 0.942187	valid_1's rmse: 1.05771
[500]	training's rmse: 0.917723	valid_1's rmse: 1.05313
[600]	training's rmse: 0.89671	valid_1's rmse: 1.04985
[700]	training's rmse: 0.875032	valid_1's rmse: 1.04788
[800]	training's rmse: 0.856539	valid_1's rmse: 1.04636
[900]	training's rmse: 0.839738	valid_1's rmse: 1.04521
[1000]	training's rmse: 0.824486	valid_1's rmse: 1.04483
[1100]	training's rmse: 0.808201	valid_1's rmse: 1.04428
[1200]	training's rmse: 0.791405	valid_1's rmse: 1.04437
[1300]	training's rmse: 0.774709	valid_1's rmse: 1.04373
[1400]	training's rmse: 0.759988	valid_1's rmse: 1.04347
[1500]	training's rmse: 0.745185	valid_1's rmse: 1.04326
[1600]	training's rmse: 0.73252	valid_1's rmse: 1.04306
[1700]	training's rmse: 0.719503	valid

D:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 150]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
D:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.05694	valid_1's rmse: 1.09955
[200]	training's rmse: 1.00184	valid_1's rmse: 1.07944
[300]	training's rmse: 0.964645	valid_1's rmse: 1.07131
[400]	training's rmse: 0.935439	valid_1's rmse: 1.06684
[500]	training's rmse: 0.910514	valid_1's rmse: 1.06387
[600]	training's rmse: 0.888704	valid_1's rmse: 1.06179
[700]	training's rmse: 0.867731	valid_1's rmse: 1.06019
[800]	training's rmse: 0.849182	valid_1's rmse: 1.05891
[900]	training's rmse: 0.831348	valid_1's rmse: 1.05816
[1000]	training's rmse: 0.813894	valid_1's rmse: 1.05737
[1100]	training's rmse: 0.796723	valid_1's rmse: 1.05705
[1200]	training's rmse: 0.780585	valid_1's rmse: 1.05691
[1300]	training's rmse: 0.767144	valid_1's rmse: 1.05676
Early stopping, best iteration is:
[1291]	training's rmse: 0.768326	valid_1's rmse: 1.05675
Predict 1/2
Valid Counts =  Counter({4: 826, 2: 791, 3: 641, 1: 608, 0: 80})
Predicted Counts =  Counter({2.0: 832,

...

Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.0583	valid_1's rmse: 1.10176
[200]	training's rmse: 1.00102	valid_1's rmse: 1.07902
[300]	training's rmse: 0.964174	valid_1's rmse: 1.06932
[400]	training's rmse: 0.933181	valid_1's rmse: 1.06349
[500]	training's rmse: 0.909477	valid_1's rmse: 1.06051
[600]	training's rmse: 0.887834	valid_1's rmse: 1.05797
[700]	training's rmse: 0.869165	valid_1's rmse: 1.0564
[800]	training's rmse: 0.850099	valid_1's rmse: 1.05525
[900]	training's rmse: 0.833294	valid_1's rmse: 1.0549
[1000]	training's rmse: 0.817729	valid_1's rmse: 1.05414
[1100]	training's rmse: 0.801453	valid_1's rmse: 1.05341
[1200]	training's rmse: 0.786387	valid_1's rmse: 1.0533
[1300]	training's rmse: 0.77108	valid_1's rmse: 1.05267
[1400]	training's rmse: 0.756847	valid_1's rmse: 1.05246
[1500]	training's rmse: 0.741567	valid_1's rmse: 1.05193
[1600]	training's rmse: 0.7278	valid_1's rmse: 1.05156
[1700]	training's rmse: 0.714223	valid_1's 

D:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 150]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
D:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.05862	valid_1's rmse: 1.10169
[200]	training's rmse: 1.00217	valid_1's rmse: 1.07636
[300]	training's rmse: 0.966179	valid_1's rmse: 1.06404
[400]	training's rmse: 0.936003	valid_1's rmse: 1.05693
[500]	training's rmse: 0.911188	valid_1's rmse: 1.05197
[600]	training's rmse: 0.889677	valid_1's rmse: 1.04864
[700]	training's rmse: 0.868674	valid_1's rmse: 1.04603
[800]	training's rmse: 0.849246	valid_1's rmse: 1.044
[900]	training's rmse: 0.830157	valid_1's rmse: 1.04229
[1000]	training's rmse: 0.813294	valid_1's rmse: 1.04113
[1100]	training's rmse: 0.797461	valid_1's rmse: 1.04073
[1200]	training's rmse: 0.780641	valid_1's rmse: 1.04036
[1300]	training's rmse: 0.766668	valid_1's rmse: 1.03997
[1400]	training's rmse: 0.753837	valid_1's rmse: 1.03986
[1500]	training's rmse: 0.740412	valid_1's rmse: 1.03969
Early stopping, best iteration is:
[1415]	training's rmse: 0.751081	valid_1's rmse: 1.0396
Pred

D:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 150]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
D:\Users\Sunfl\Anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.05844	valid_1's rmse: 1.09772
[200]	training's rmse: 1.0019	valid_1's rmse: 1.07285
[300]	training's rmse: 0.966038	valid_1's rmse: 1.06357
[400]	training's rmse: 0.936489	valid_1's rmse: 1.05817
[500]	training's rmse: 0.91168	valid_1's rmse: 1.05461
[600]	training's rmse: 0.886319	valid_1's rmse: 1.05183
[700]	training's rmse: 0.864628	valid_1's rmse: 1.04956
[800]	training's rmse: 0.844688	valid_1's rmse: 1.04766
[900]	training's rmse: 0.825881	valid_1's rmse: 1.04658
[1000]	training's rmse: 0.808867	valid_1's rmse: 1.04543
[1100]	training's rmse: 0.791425	valid_1's rmse: 1.04446
[1200]	training's rmse: 0.775055	valid_1's rmse: 1.0437
[1300]	training's rmse: 0.757944	valid_1's rmse: 1.0431
[1400]	training's rmse: 0.741408	valid_1's rmse: 1.04224
[1500]	training's rmse: 0.726465	valid_1's rmse: 1.04162
[1600]	training's rmse: 0.712155	valid_1's rmse: 1.04133
[1700]	training's rmse: 0.697742	valid_1

In [16]:
imports = results['importance'].groupby('feature')['feature', 'importance'].mean().reset_index()
imports.sort_values('importance', ascending=False)

,feature,importance
1,Breed1,1898.8
0,Age,992.6
2,Breed2,705.6
12,PhotoAmt,566.4
14,State,554.0
91,svd_46,493.0
99,svd_53,470.4
85,svd_40,439.8
13,Quantity,436.4
30,svd_0,431.8


In [17]:
optR = OptimizedRounder()
coefficients_ = np.mean(results['coefficients'], axis=0)
print(coefficients_)
# manually adjust coefs
coefficients_[0] = 1.64
coefficients_[1] = 2.11
coefficients_[3] = 2.85
train_predictions = [r[0] for r in results['train']]
train_predictions = optR.predict(train_predictions, coefficients_).astype(int)
Counter(train_predictions)

[0.50370567 1.82166141 2.52838151 2.85153794]


Counter({2: 4368, 1: 2903, 3: 2910, 4: 3957, 0: 582})

In [18]:
optR = OptimizedRounder()
coefficients_ = np.mean(results['coefficients'], axis=0)
print(coefficients_)
# manually adjust coefs
coefficients_[0] = 1.645
coefficients_[1] = 2.115
coefficients_[3] = 2.84
test_predictions = [r[0] for r in results['test']]
test_predictions = optR.predict(test_predictions, coefficients_).astype(int)
Counter(test_predictions)

[0.50370567 1.82166141 2.52838151 2.85153794]


Counter({2: 1175, 4: 1120, 3: 766, 1: 803, 0: 84})

In [19]:
print("True Distribution:")
print(pd.value_counts(target, normalize=True).sort_index())
print("Test Predicted Distribution:")
print(pd.value_counts(test_predictions, normalize=True).sort_index())
print("Train Predicted Distribution:")
print(pd.value_counts(train_predictions, normalize=True).sort_index())

True Distribution:
0    0.026970
1    0.206386
2    0.268682
3    0.217527
4    0.280435
Name: AdoptionSpeed, dtype: float64
Test Predicted Distribution:
0    0.021277
1    0.203394
2    0.297619
3    0.194022
4    0.283688
dtype: float64
Train Predicted Distribution:
0    0.039538
1    0.197215
2    0.296739
3    0.197690
4    0.268818
dtype: float64


In [20]:
pd.DataFrame(sk_cmatrix(target, train_predictions), index=list(range(5)), columns=list(range(5)))

,0,1,2,3,4
0,65,118,103,49,62
1,279,1023,995,463,278
2,150,988,1436,736,645
3,68,533,1073,786,742
4,20,241,761,876,2230


In [21]:
quadratic_weighted_kappa(target, train_predictions)
rmse(target, [r[0] for r in results['train']])
submission = pd.DataFrame({'PetID': test_id, 'AdoptionSpeed': test_predictions})
submission.head()

,PetID,AdoptionSpeed
0,378fcc4fc,2
1,73c10e136,4
2,72000c4c5,4
3,e147a4b9f,3
4,43fbba852,4


In [22]:
submission.to_csv('submission.csv', index=False)

In [23]:
print("QWK =  0.41241364058533925")

QWK =  0.41241364058533925
